In [1]:
raw_corpus_path = 'data/corpus_10days/news/2016-10-24_article_all.txt'
norm_corpus_path = 'tmp/norm_2016-10-24_article_all.txt'

# DO_NORMALIZE = False

# max_l_length = 8
# max_r_length = 5
# min_count = 30

# noun_score_threshold = 0.2

from collections import defaultdict
import numpy as np
import pickle
import sys

from corpus import Corpus
corpus = Corpus(norm_corpus_path, iter_sent=False)

with open('tmp/extracted_noun_dict.pkl', 'rb') as f:
    noun_dict = pickle.load(f)

In [2]:
type(noun_dict)

dict

In [3]:
noun_dict

{'재원': 0.5867644615384615,
 '선임기자': 0.9688135,
 '외관': 0.6792262375,
 '우호': 0.675858,
 '아이오아': 0.9337285081967214,
 '노벨': 0.99883,
 '거래가': 0.578128,
 '안전보장이사회': 0.9993221304347826,
 '유명': 0.9975225527777778,
 '목표가': 0.9632535512820513,
 '길거리': 0.9637801428571429,
 '2TV': 0.9996628214285713,
 '벌써': 1.0,
 '일시적으로': 0.42971000000000004,
 '최대한': 0.9833394999999999,
 '패션위크': 0.6555941666666667,
 '평양': 0.8694437692307693,
 '요동': 0.9422424590163935,
 '3D': 0.9435263529411766,
 '계약금': 0.3200045714285714,
 '기호': 0.9898051578947369,
 '적극적으': 0.99883,
 '신혼부부': 0.8802432307692308,
 '입담으': 0.9988300000000001,
 '패러다임': 0.6225608823529412,
 '중복': 0.9996687837837839,
 '주요종목들': 0.999913,
 '특위': 0.7396546951219513,
 '재건축': 0.8485515161290322,
 '비화': 0.8954438333333332,
 '훈훈': 0.9979459247311828,
 '제3자': 0.9988983571428571,
 '불편': 0.8649028216783216,
 '사냥': 0.7603517777777777,
 '서초구': 0.3352957777777778,
 '언니': 0.7538476014492754,
 '카운트다운': 0.9997200000000002,
 '영하로': 0.99961,
 '지창욱': 0.1689081724137931,
 

## Term frequency matrix 만들기

In [4]:
def custom_tokenize(doc):
    
    def parse_noun(token):
        for e in reversed(range(1, len(token)+1)):
            subword = token[:e]
            if subword in noun_dict:
                return subword
        return ''
    
    nouns = [parse_noun(token) for token in doc.split()]
    nouns = [word for word in nouns if word]
    return nouns

문서가 주어지면 추출된 명사만을 출력하는 custom_tokenize를 만들었습니다. 이를 이용하면 tokenize, part of speech tagging, 이후 명사만 추출하는 과정을 거친 것과 같습니다. 

In [5]:
corpus.iter_sent = True

for num_sent, sent in enumerate(corpus):
    
    if num_sent == 5:
        break
        
    print(sent, '\n')
    print(custom_tokenize(sent), '\n\n')

NGO 의원 60명 내무장관에게 서한 철거 때 어린이 안전 신경 써야 

['NG', '의', '60명', '서한', '철거', '때', '어린이', '안전', '신경', '써야'] 


보호자 없는 아동 난민 70명은 영국에 도착 

['보호자', '아동', '난민', '영국', '도착'] 


22일 경찰에 돌 던지는 칼레 난민 EPA 연합뉴스 

['22일', '경찰', '칼레', '난민', 'EPA', '연합뉴스'] 


파리 런던 연합뉴스 박성진 황정우 특파원 프랑스 칼레 난민촌 철거를 이틀 앞둔 22일 현지시간 난민촌 주변에서 현지 경찰과 난민이 충돌했다고 현지 TV 프랑스앵포가 보도했다 

['파리', '런던', '연합뉴스', '특파원', '프랑스', '칼레', '난민촌', '철거', '이틀', '22일', '현지시간', '난민촌', '주변에서', '현지', '경찰과', '난민', '충돌', '현지', 'TV', '프랑스', '보도했다'] 


난민촌 철거에 반대하는 난민 50명가량은 경찰을 향해 유리병과 돌을 던졌으며 경찰은 연막탄을 쏘면서 이들을 해산했다 

['난민촌', '철거', '반대', '난민', '50명', '경찰', '향', '유리', '경찰', '이들', '해산'] 




반드시 작성한 코드에 문제가 없는지 확인을 합니다

In [6]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(tokenizer=custom_tokenize)
x = vectorizer.fit_transform(corpus)

CountVectorizer를 이용하여 문서를 term frequency matrix로 만들었습니다

## Classification을 위한 positive / negative set 만들기

In [7]:
aspect_word = '국회'
aspect_id = vectorizer.vocabulary_.get(aspect_word, -1)

print('%s = %d' % (aspect_word, aspect_id))
print('num of doc (%s) = %d' % (aspect_word, len(x[:,aspect_id].nonzero()[0])))

국회 = 2207
num of doc (국회) = 6382


In [8]:
print(x[:,aspect_id].shape)

from pprint import pprint
pprint(x[:,aspect_id].nonzero())

(221674, 1)
(array([   196,    197,    207, ..., 221640, 221644, 221653], dtype=int32),
 array([0, 0, 0, ..., 0, 0, 0], dtype=int32))


sparse matrix는 nonzero()라는 함수가 있으며, matrix에서 값이 0이 아닌 위치를 (row id list, column id list)로 나타내줍니다. aspect_id에 해당하는 컬럼을 떼어냈기 때문에 submatrix의 모양이 (221674, 1)이 되었습니다

In [9]:
pos_idx = set(x[:,aspect_id].nonzero()[0]) # in 함수는 list보다 set이 빠릅니다
y = [1 if i in pos_idx else -1 for i in range(x.shape[0])]

print('x shape = %s, len(y) = %d, num_pos = %d' % (str(x.shape), len(y), len(pos_idx)))

x shape = (221674, 13643), len(y) = 221674, num_pos = 6382


In [10]:
pos_idx

{163840,
 98386,
 98390,
 98391,
 98393,
 98394,
 163939,
 163940,
 163941,
 98410,
 163946,
 98413,
 65646,
 98424,
 32889,
 65659,
 98427,
 98428,
 98438,
 98449,
 98455,
 98456,
 98457,
 98458,
 98460,
 196771,
 131241,
 131243,
 131258,
 131259,
 131261,
 196,
 197,
 32964,
 32966,
 32967,
 32968,
 131273,
 65741,
 207,
 209,
 216,
 217,
 218,
 219,
 65756,
 65759,
 65763,
 131299,
 164072,
 164073,
 98539,
 65772,
 98540,
 65778,
 131329,
 33027,
 33030,
 131339,
 65804,
 65805,
 131340,
 65807,
 65808,
 131345,
 164107,
 164108,
 164116,
 164132,
 98597,
 98598,
 98599,
 98600,
 164133,
 164134,
 164135,
 164141,
 164142,
 98609,
 98610,
 98611,
 309,
 98613,
 164158,
 164159,
 98625,
 98626,
 98627,
 33092,
 98637,
 98639,
 98640,
 33125,
 33129,
 33130,
 33133,
 33134,
 65910,
 65911,
 33155,
 65956,
 33191,
 197086,
 66021,
 66025,
 66028,
 66030,
 66032,
 66037,
 512,
 164364,
 164366,
 33347,
 33348,
 33351,
 66131,
 33365,
 66135,
 33368,
 33372,
 33373,
 33379,
 33386,
 16

In [11]:
y

[-1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 -1,
 1,
 1,
 -1,
 -1,
 -

'국회'란 단어가 들어간 문서들의 리스트를 sparse matrix에서 가져왔습니다. 이를 이용하여 y_label을 만듦니다

## Logistic regression을 이용한 문서 분류기

#### LogisticRegression의 constructor

## L2 regulization을 이용한 문서 판별기 학습

In [12]:
from sklearn.linear_model import LogisticRegression
LogisticRegression?

In [13]:
from sklearn.linear_model import LogisticRegression

logistic_l2 = LogisticRegression(penalty='l2')
logistic_l2.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [14]:
y_pred = logistic_l2.predict(x)
print(y_pred.shape)

(221674,)


In [15]:
accuracy = sum([1 if pred == answ else 0 for pred, answ in zip(y_pred, y)]) / len(y_pred)
print('training error = %.3f' % (1 - accuracy))

training error = 0.000


위 정확도는 training error를 보기 위함이며, test set에 의한 일반화 성능 및 (Cross validation)은 나중에 다루도록 하겠습니다. 

## regression coefficients 뜯어보기

In [16]:
print('logistic regression coefficient:',logistic_l2.coef_.shape)
print('size of vocabulary:', len(vectorizer.vocabulary_))

logistic regression coefficient: (1, 13643)
size of vocabulary: 13643


vectorizer의 vocabulary_의 크기와 logistic_l2.coef_의 크기가 같습니다. coef는 각 단어의 positive class에 대한 기여도입니다. 

In [17]:
logistic_l2.coef_.reshape(-1).shape

(13643,)

matrix.reshape(-1)을 하면 matrix를 vector 형태로 바꿔줍니다

In [18]:
for i, v in enumerate(logistic_l2.coef_.reshape(-1)):
    print(i, v)
    if i == 3: break

0 -0.103918485135
1 0.0546020662582
2 -0.00176869636261
3 0.00406840827199


In [19]:
index2word = sorted(vectorizer.vocabulary_.items(), key=lambda x:x[1])
index2word = [word for word, i in index2word]
print(index2word[50:55])

['10년', '10년간', '10대', '10도', '10만명']


In [20]:
coef_l2 = logistic_l2.coef_.reshape(-1)
beta_l2 = [(index2word[i], coef) for i, coef in enumerate(coef_l2)]
beta_l2 = sorted(beta_l2, key=lambda x:x[1], reverse=True)

In [21]:
beta_l2[:50]

[('국회', 13.456574566299283),
 ('시정연설', 0.75747178712066499),
 ('여의도', 0.62025855332442492),
 ('예산안', 0.54198869661214166),
 ('20대', 0.53897453097257109),
 ('국정감사', 0.51921792842238534),
 ('시정연설에서', 0.41803455534515971),
 ('최고위원회의', 0.3843862204472962),
 ('예산', 0.38297191375844214),
 ('국민과', 0.34389819602017541),
 ('대통령', 0.31242898738657027),
 ('전체회의', 0.30390116899046909),
 ('오전', 0.30325526987913004),
 ('구성', 0.29791698239773745),
 ('여야', 0.29352611495708442),
 ('더불어민주당', 0.29179136427940361),
 ('의원총회', 0.28838938741066594),
 ('박근혜', 0.28716283391234304),
 ('2017년도', 0.28269043881902883),
 ('본회의장', 0.28105619367686685),
 ('24일', 0.26424386306333214),
 ('정부', 0.25022762679136684),
 ('새누리당', 0.24949784116060247),
 ('24', 0.24947399051839381),
 ('통과', 0.24820843034970336),
 ('의결', 0.23467345590094391),
 ('이같', 0.23409537448805456),
 ('헌법개정', 0.23164639250777974),
 ('개헌안', 0.22942047440701682),
 ('논의', 0.22657165585067618),
 ('개헌특위', 0.22289848910427706),
 ('개헌', 0.21746809295216737),
 (

## L1 (LASSO)를 이용한 keyword extraction

In [27]:
from sklearn.linear_model import LogisticRegression

logistic_l1 = LogisticRegression(penalty='l1', random_state=1234)
logistic_l1.fit(x, y)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=1234, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [29]:
coef_l1 = logistic_l1.coef_.reshape(-1)
beta_l1 = [(index2word[i], coef) for i, coef in enumerate(coef_l1)]
beta_l1 = sorted(beta_l1, key=lambda x:x[1], reverse=True)
beta_l1 = [(word, coef) for word, coef in beta_l1 if coef > 0.001]

beta_l1[:50]

[('국회', 19.627472786016806)]

### L1 cost에 따른 keyword 변화

In [30]:
costs = [500, 200, 100, 50]
top50 = {} 

for cost in costs:
    logistic_l1 = LogisticRegression(penalty='l1', C=cost, random_state=1234)
    logistic_l1.fit(x, y)
    
    coef_l1 = logistic_l1.coef_.reshape(-1)
    beta_l1 = [(index2word[i], coef) for i, coef in enumerate(coef_l1)]
    beta_l1 = sorted(beta_l1, key=lambda x:x[1], reverse=True)
    top50[cost] = beta_l1[:50]
    print('done with cost = %.1f' % cost)

done with cost = 500.0
done with cost = 200.0
done with cost = 100.0
done with cost = 50.0


In [31]:
for top in range(50):
    message = '\t'.join(['%15s' % (top50[cost][top][0] if top50[cost][top][1] > 0.001 else '') for cost in costs])
    print(message)
        

             국회	             국회	             국회	             국회
          성공패키지	           조선일보	           시정연설	            대통령
             본지	           국정감사	            근로자	            보조금
           조선일보	             구호	            여의도	            근로자
           국정감사	           시정연설	            보조금	             사업
            여의도	           사회보험	           국민연금	               
           시정연설	          성공패키지	            예산안	               
            사업주	            국민과	            보험료	               
         시정연설에서	         최고위원회의	            사업장	               
            보조금	            여의도	            대통령	               
         최고위원회의	           두루누리	           새누리당	               
           지원사업	           전체회의	             개헌	               
             구호	           국민들이	             대표	               
            국민과	             본지	           국정감사	               
            20대	           세아상역	             예산	               
             산재	            20대	        